# <span style="font-size:23pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">Bryan Sanchez - SQL NOIR Mystery Cases</span>

<span style="font-size: 10.5pt; font-family: Arial, sans-serif; color: rgb(0, 0, 0); background-color: rgb(255, 255, 255); font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; font-variant-position: normal; vertical-align: baseline; white-space-collapse: preserve;">*used white highlight for text to see it on vscode</span>

<span style="font-size:17pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">Case #1: The Van Heist</span>

<span style="font-size:11pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">Target:</span><span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: Arial, sans-serif; font-size: 10.5pt; white-space-collapse: preserve;">Find out which delivery went missing.You’ll look for invoices connected to a specific delivery method (like “Delivery Van”) and see who was responsible for it.</span>

<span style="font-size:11pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">Objectives: </span>   

<span style="font-size:11pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">1)</span><span style="font-size:10.5pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">Get Delivery Van’s ID</span>

<span style="font-size:11pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">2)</span><span style="font-size:10.5pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">Among van-shipped invoices, compute line‑ level discount vs catalog and pick the highest</span>

<span style="font-size: 11pt; font-family: Arial, sans-serif; color: rgb(0, 0, 0); background-color: rgb(255, 255, 255); font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; font-variant-position: normal; vertical-align: baseline; white-space-collapse: preserve;">3)</span><span style="font-size: 10.5pt; font-family: Arial, sans-serif; color: rgb(0, 0, 0); background-color: rgb(255, 255, 255); font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; font-variant-position: normal; vertical-align: baseline; white-space-collapse: preserve;">Identify the salesperson</span>

In [ ]:
-- CLUE 1 DeliveryMethodID for 'Delivery Van' (Application.DeliveryMethods)
USE WideWorldImporters;
SELECT DeliveryMethodID
FROM Application.DeliveryMethods
WHERE DeliveryMethodName = N'Delivery Van';

In [ ]:
--CLUE 2 Largest discount using that method (Sales.Invoices + Sales.InvoiceLines + Warehouse.StockItems)
DECLARE @VanID int = (
 SELECT DeliveryMethodID FROM Application.DeliveryMethods WHERE
DeliveryMethodName = N'Delivery Van'
);
WITH Deals AS (
 SELECT i.InvoiceID, i.InvoiceDate, i.SalespersonPersonID,
 CAST((si.UnitPrice - il.UnitPrice)/NULLIF(si.UnitPrice,0.0) AS
decimal(10,4)) AS DiscRatio
 FROM Sales.Invoices i
 JOIN Sales.InvoiceLines il ON il.InvoiceID = i.InvoiceID
 JOIN Warehouse.StockItems si ON si.StockItemID = il.StockItemID
 WHERE i.DeliveryMethodID = @VanID
 AND i.InvoiceDate >= DATEADD(day,-60,(SELECT MAX(InvoiceDate) FROM
Sales.Invoices))
)
SELECT TOP (1) * FROM Deals ORDER BY DiscRatio DESC, InvoiceDate DESC;

In [ ]:
-- CLUE 3 Salesperson’s name (Application.People)
SELECT FullName AS Salesperson
FROM Application.People
WHERE PersonID = 8;

<span style="font-size:17pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">Case #2: The Ghost Orders</span>

<span style="font-size:11pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">Target:</span><span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: Arial, sans-serif; font-size: 10.5pt; white-space-collapse: preserve;">Find customers who placed orders but never got an invoice.You’ll check which customer has the most unbilled orders and which salesperson handled them.</span>

<span style="font-size:10.5pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">Objectives</span>

<span style="font-size:10.5pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">1) Find orders without invoices</span>

<span style="font-size:10.5pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">2) Name the top customer</span>

<span style="font-size:10.5pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">3) Find the rep linked to those misses</span>

In [ ]:
-- CLUE 1 Orders missing invoices (from Sales.Orders to Sales.Invoices)
DECLARE @MaxOrderDate date = (SELECT MAX(OrderDate) FROM Sales.Orders);

SELECT TOP (1) CustomerID, COUNT(*) AS MissingCount
FROM (
  SELECT o.OrderID, o.CustomerID, o.SalespersonPersonID
  FROM Sales.Orders o
  LEFT JOIN Sales.Invoices i ON i.OrderID = o.OrderID
  WHERE i.InvoiceID IS NULL
    AND o.OrderDate >= DATEADD(day,-30,@MaxOrderDate)
) AS NoInvoice
GROUP BY CustomerID
ORDER BY MissingCount DESC;


In [ ]:
-- CLUE 2 get Customer name (Sales.Customers)
SELECT CustomerID, CustomerName
FROM Sales.Customers
WHERE CustomerID = 153;


In [ ]:
-- CLUE 3 Rep with most misses (Application.People)
DECLARE @CustomerID int = 153;

SELECT TOP (1)
       ISNULL(p.FullName,'(Unassigned)') AS Salesperson,
       COUNT(*) AS Missed
FROM (
  SELECT o.OrderID, o.CustomerID, o.SalespersonPersonID
  FROM Sales.Orders o
  LEFT JOIN Sales.Invoices i ON i.OrderID = o.OrderID
  WHERE i.InvoiceID IS NULL
    AND o.OrderDate >= DATEADD(day,-30,(SELECT MAX(OrderDate) FROM Sales.Orders))
    AND o.CustomerID = @CustomerID
) AS NoInvoice
LEFT JOIN Application.People p ON p.PersonID = NoInvoice.SalespersonPersonID
GROUP BY ISNULL(p.FullName,'(Unassigned)')
ORDER BY Missed DESC, Salesperson;

<span style="font-size: 17pt; font-family: Arial, sans-serif; color: rgb(0, 0, 0); background-color: rgb(255, 255, 255); font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; font-variant-position: normal; vertical-align: baseline; white-space-collapse: preserve;">Case #3: Selling On Empty</span>

<span style="font-size: 11pt; font-family: Arial, sans-serif; color: rgb(0, 0, 0); background-color: rgb(255, 255, 255); font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; font-variant-position: normal; vertical-align: baseline; white-space-collapse: preserve;">Target:</span><span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: Arial, sans-serif; font-size: 10.5pt; white-space-collapse: preserve;">Find which items are being sold even though the stock is too low.Then, track which supplier provides that product.</span>

<span style="font-size: 10.5pt; font-family: Arial, sans-serif; color: rgb(0, 0, 0); background-color: rgb(255, 255, 255); font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; font-variant-position: normal; vertical-align: baseline; white-space-collapse: preserve;">Objectives</span>

<span style="font-size: 10.5pt; font-family: Arial, sans-serif; color: rgb(0, 0, 0); background-color: rgb(255, 255, 255); font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; font-variant-position: normal; vertical-align: baseline; white-space-collapse: preserve;">1) List Low-Stock Items</span>

<span style="font-size: 10.5pt; font-family: Arial, sans-serif; color: rgb(0, 0, 0); background-color: rgb(255, 255, 255); font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; font-variant-position: normal; vertical-align: baseline; white-space-collapse: preserve;">2) See if any sold this week</span>

<span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: Arial, sans-serif; font-size: 10.5pt; white-space-collapse: preserve;">3)</span><span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: Arial, sans-serif; font-size: 10.5pt; white-space-collapse: preserve;">Find the supplier for one such item</span>

In [ ]:
-- CLUE 1 Low stock items (Warehouse.StockItemHoldings + Warehouse.StockItems)
SELECT TOP (10) *
FROM (
  SELECT si.StockItemID, si.StockItemName, sh.QuantityOnHand, sh.ReorderLevel,
         (sh.ReorderLevel - sh.QuantityOnHand) AS Deficit
  FROM Warehouse.StockItemHoldings sh
  JOIN Warehouse.StockItems si ON si.StockItemID = sh.StockItemID
  WHERE sh.QuantityOnHand < sh.ReorderLevel
) AS Low
ORDER BY Deficit DESC;

In [ ]:
-- CLUE 2 Recent sales for one candidate (Sales.InvoiceLines + Sales.Invoices)
DECLARE @ItemID int = 184;
SELECT TOP (1) il.InvoiceID, i.InvoiceDate
FROM Sales.InvoiceLines il JOIN Sales.Invoices i ON i.InvoiceID = il.InvoiceID
WHERE il.StockItemID = @ItemID
 AND i.InvoiceDate >= DATEADD(day,-7,(SELECT MAX(InvoiceDate) FROM
Sales.Invoices))
ORDER BY i.InvoiceDate DESC;


In [ ]:
-- CLUE 3 Supplier (Purchasing.Suppliers)
SELECT s.SupplierName
FROM Purchasing.Suppliers s
JOIN Warehouse.StockItems si ON si.SupplierID = s.SupplierID
WHERE si.StockItemID = @ItemID;


<span style="font-size: 17pt; font-family: Arial, sans-serif; color: rgb(0, 0, 0); background-color: rgb(255, 255, 255); font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; font-variant-position: normal; vertical-align: baseline; white-space-collapse: preserve;">Case #4: Purchasing Order Similarity</span>

<span style="font-size: 11pt; font-family: Arial, sans-serif; color: rgb(0, 0, 0); background-color: rgb(255, 255, 255); font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; font-variant-position: normal; vertical-align: baseline; white-space-collapse: preserve;">Target:</span><span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: Arial, sans-serif; font-size: 11pt; white-space-collapse: preserve;">Find customers who reused the same purchase order number more than once.Then, check which invoice was created with that repeated order.</span>

<span style="font-size: 10.5pt; font-family: Arial, sans-serif; color: rgb(0, 0, 0); background-color: rgb(255, 255, 255); font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; font-variant-position: normal; vertical-align: baseline; white-space-collapse: preserve;">Objectives</span>

<span style="font-size: 10.5pt; font-family: Arial, sans-serif; color: rgb(0, 0, 0); background-color: rgb(255, 255, 255); font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; font-variant-position: normal; vertical-align: baseline; white-space-collapse: preserve;">1) Detect duplicate customer PO numbers</span>

<span style="font-size: 10.5pt; font-family: Arial, sans-serif; color: rgb(0, 0, 0); background-color: rgb(255, 255, 255); font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; font-variant-position: normal; vertical-align: baseline; white-space-collapse: preserve;">2) Confirm at least one became an invoice</span>

In [ ]:
-- CLUE 1 Duplicate POs (Sales.Orders)
USE WideWorldImporters;

DECLARE @MaxOrderDate date = (SELECT MAX(OrderDate) FROM Sales.Orders);

SELECT TOP (1)
       o.CustomerID,
       o.CustomerPurchaseOrderNumber AS PO,
       COUNT(*) AS Uses
FROM Sales.Orders AS o
WHERE o.CustomerPurchaseOrderNumber IS NOT NULL
  AND o.OrderDate >= DATEADD(day, -180, @MaxOrderDate)
GROUP BY o.CustomerID, o.CustomerPurchaseOrderNumber
HAVING COUNT(*) > 1
ORDER BY Uses DESC, o.CustomerPurchaseOrderNumber;


In [ ]:
-- CLUE 2 Invoice trail (Sales.Invoices)
DECLARE @CustomerID int = 65
DECLARE @PO nvarchar(50) = 10097

SELECT TOP (1)
       i.InvoiceID,
       i.InvoiceDate,
       c.CustomerName
FROM Sales.Invoices AS i
JOIN Sales.Orders   AS o ON o.OrderID = i.OrderID
JOIN Sales.Customers AS c ON c.CustomerID = o.CustomerID
WHERE o.CustomerID = @CustomerID
  AND o.CustomerPurchaseOrderNumber = @PO
ORDER BY i.InvoiceDate;

<span style="font-size:17pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">Case #5: The Slowest Lane</span>

<span style="font-size:11pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">Target: Find which delivery method takes the longest time between order and invoice. Then, find the most recent customer who experienced this delay.</span>

<span style="font-size:10.5pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">Objectives</span>

<span style="font-size:10.5pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">1)</span><span style="font-size:11pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">Determine which delivery method has the highest average delay between the order date and the invoice date.</span>

<span style="font-size: 10.5pt; font-family: Arial, sans-serif; color: rgb(0, 0, 0); background-color: rgb(255, 255, 255); font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; font-variant-position: normal; vertical-align: baseline; white-space-collapse: preserve;">2)I</span><span style="font-size: 11pt; font-family: Arial, sans-serif; color: rgb(0, 0, 0); background-color: rgb(255, 255, 255); font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; font-variant-position: normal; vertical-align: baseline; white-space-collapse: preserve;">dentify the most recent invoice that used this method and reveal the customer associated with that delayed delivery.</span>

In [ ]:
-- CLUE 1 Find the slowest delivery method
SELECT TOP (1) dm.DeliveryMethodID, dm.DeliveryMethodName, L.AvgDays
FROM (
  SELECT i.DeliveryMethodID,
         AVG(DATEDIFF(day,o.OrderDate,i.InvoiceDate)*1.0) AS AvgDays
  FROM Sales.Orders o
  JOIN Sales.Invoices i ON i.OrderID = o.OrderID
  GROUP BY i.DeliveryMethodID
) AS L
JOIN Application.DeliveryMethods dm ON dm.DeliveryMethodID = L.DeliveryMethodID
ORDER BY L.AvgDays DESC, dm.DeliveryMethodName;


In [ ]:
-- CLUE 2 Find the most recent invoice that used this slow delivery method
DECLARE @Slow int = 3 ;

SELECT TOP (1) i.InvoiceID, i.InvoiceDate, c.CustomerName
FROM Sales.Invoices i
JOIN Sales.Customers c ON c.CustomerID = i.CustomerID
WHERE i.DeliveryMethodID = @Slow
ORDER BY i.InvoiceDate DESC, i.InvoiceID DESC;

<span style="font-size:17pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">Case #6: The Finalized but late supplier</span>

<span style="font-size:11pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">Target: Find suppliers who often finalize purchase orders but deliver late. You’ll identify which delivery method is most common among these late suppliers.</span>

<span style="font-size:11pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">Objectives</span>

<span style="font-size:11pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">1) Count overdue finalized POs by supplier</span>

<span style="font-size:11pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">2) Show their common ship method</span>

In [ ]:
--CLUE 1 Overdue finalized POs (Purchasing.PurchaseOrders)
SELECT TOP (1) SupplierID, COUNT(*) AS LateCount
FROM (
  SELECT SupplierID
  FROM Purchasing.PurchaseOrders
  WHERE IsOrderFinalized = 1
    AND ExpectedDeliveryDate < GETDATE()
) AS LatePOs
GROUP BY SupplierID
ORDER BY LateCount DESC;

In [ ]:
-- CLUE 2 Ship method (Application.DeliveryMethods)
DECLARE @Supp int = 4;

SELECT TOP (1) dm.DeliveryMethodName, COUNT(*) AS Uses
FROM Purchasing.PurchaseOrders po
JOIN Application.DeliveryMethods dm ON dm.DeliveryMethodID = po.DeliveryMethodID
WHERE po.IsOrderFinalized = 1
  AND po.ExpectedDeliveryDate < GETDATE()
  AND po.SupplierID = @Supp
GROUP BY dm.DeliveryMethodName
ORDER BY Uses DESC;

<span style="font-size:17pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">Case #7: Picker on the run</span>

<span style="font-size:11pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">Target: Find out which picker handles the most urgent orders (orders with short delivery times). Then, check which recent orders that picker completed.</span>

<span style="font-size:10.5pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">Objectives</span>

<span style="font-size:10.5pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">1)</span><span style="font-size:11pt;font-family:Arial,sans-serif;color:#000000;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">Filter urgent orders</span>

<span style="font-size: 10.5pt; font-family: Arial, sans-serif; color: rgb(0, 0, 0); background-color: rgb(255, 255, 255); font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; font-variant-position: normal; vertical-align: baseline; white-space-collapse: preserve;">2)</span><span style="font-size: 11pt; font-family: Arial, sans-serif; color: rgb(0, 0, 0); background-color: rgb(255, 255, 255); font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; font-variant-position: normal; vertical-align: baseline; white-space-collapse: preserve;">Count by picker and show a recent order</span>

In [ ]:
--CLUE 1 Top picker on urgent orders
SELECT TOP (1) PickedByPersonID, COUNT(*) AS UrgentPicks
FROM (
    SELECT OrderID, PickedByPersonID
    FROM Sales.Orders
    WHERE PickedByPersonID IS NOT NULL
      AND DATEDIFF(day, OrderDate, ExpectedDeliveryDate) <= 2
      AND OrderDate >= DATEADD(day, -45, @MaxOrderDate)
) AS Urgent
GROUP BY PickedByPersonID
ORDER BY UrgentPicks DESC;

In [ ]:
--CLUE 2 Show Example order + name
DECLARE @Picker int = 10;

SELECT FullName AS Picker
FROM Application.People
WHERE PersonID = @Picker;

SELECT TOP (1) o.OrderID, o.OrderDate, o.ExpectedDeliveryDate
FROM Sales.Orders o
WHERE o.PickedByPersonID = @Picker
  AND DATEDIFF(day, o.OrderDate, o.ExpectedDeliveryDate) <= 2
ORDER BY o.OrderDate DESC, o.OrderID DESC;

\*<span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: Arial, sans-serif; font-size: 11pt; white-space-collapse: preserve;">AI was used in the creation of this assignment, and all work was reviewed and finalized by our team</span>